In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
input_size=784
hidden_size=500
num_classes=10
num_epochs=5
batch_size=100
learning_rate=0.001

In [4]:
from torch.utils.data import dataset
train_dataset=torchvision.datasets.MNIST(root='inputdata',train=True,transform=transforms.ToTensor(),download=True)
test_dataset=torchvision.datasets.MNIST(root='inputdata',train=False,transform=transforms.ToTensor())

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting inputdata/MNIST/raw/train-images-idx3-ubyte.gz to inputdata/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting inputdata/MNIST/raw/train-labels-idx1-ubyte.gz to inputdata/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting inputdata/MNIST/raw/t10k-images-idx3-ubyte.gz to inputdata/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting inputdata/MNIST/raw/t10k-labels-idx1-ubyte.gz to inputdata/MNIST/raw



In [17]:
from torch.utils.data import DataLoader

train_loader=DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader=DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=False)

In [31]:
class NeuralNet(nn.Module):
  def __init__(self,input_size,hidden_size,num_classes) -> None:
      super(NeuralNet,self).__init__()
      self.fc1=nn.Linear(input_size,hidden_size)
      self.relu=nn.ReLU()
      self.fc2=nn.Linear(hidden_size,num_classes)
      
  def forward(self,x):
    out=self.fc1(x)
    out=self.relu(out)
    out=self.fc2(out)
    return out



In [34]:
import torch.optim as optim

model=NeuralNet(input_size,hidden_size,num_classes).to(device)
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

total_step=len(train_loader)
for a in range(num_epochs):
  for i,(images,labels) in enumerate(train_loader):
    images=images.reshape(-1,28*28).to(device)
    labels=labels.to(device)

    outputs=model(images)
    loss=criterion(outputs,labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 100 == 0:
      print('Epoch [{}/{}] , Step [{}/{}] , Loss {:.4f}'.format(a+1,num_epochs,i+1,total_step,loss.item()))

Epoch [1/5] , Step [100/600] , Loss 0.2747
Epoch [1/5] , Step [200/600] , Loss 0.3153
Epoch [1/5] , Step [300/600] , Loss 0.1881
Epoch [1/5] , Step [400/600] , Loss 0.2226
Epoch [1/5] , Step [500/600] , Loss 0.0634
Epoch [1/5] , Step [600/600] , Loss 0.2009
Epoch [2/5] , Step [100/600] , Loss 0.0747
Epoch [2/5] , Step [200/600] , Loss 0.1695
Epoch [2/5] , Step [300/600] , Loss 0.0452
Epoch [2/5] , Step [400/600] , Loss 0.2101
Epoch [2/5] , Step [500/600] , Loss 0.1008
Epoch [2/5] , Step [600/600] , Loss 0.0293
Epoch [3/5] , Step [100/600] , Loss 0.0524
Epoch [3/5] , Step [200/600] , Loss 0.0630
Epoch [3/5] , Step [300/600] , Loss 0.1659
Epoch [3/5] , Step [400/600] , Loss 0.0877
Epoch [3/5] , Step [500/600] , Loss 0.0689
Epoch [3/5] , Step [600/600] , Loss 0.1065
Epoch [4/5] , Step [100/600] , Loss 0.0428
Epoch [4/5] , Step [200/600] , Loss 0.0479
Epoch [4/5] , Step [300/600] , Loss 0.0617
Epoch [4/5] , Step [400/600] , Loss 0.0572
Epoch [4/5] , Step [500/600] , Loss 0.0647
Epoch [4/5]

In [36]:
with torch.no_grad():
  correct=0
  total=0
  for images,labels in test_loader:
    images=images.reshape(-1,28*28).to(device)
    labels=labels.to(device)
    outputs=model(images)
    _ ,predicted=torch.max(outputs.data,1)
    total+=labels.size(0)
    correct+=(predicted==labels).sum().item()
  print('accuracy:{} %'.format( 100* correct/total))

accuracy:99.17333333333333 %
